**DAY9: Netflix https://lnkd.in/guCXi8k9 Find the genre of the person with the most number of oscar winnings.
If there are more than one person with the same number of oscar wins, return the first one in alphabetic order based on their name. Use the names as keys when joining the tables.**

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/nominee_information", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/oscar_nominees", True)

Out[7]: True

In [0]:
%sql
DROP TABLE IF EXISTS nominee_information;
DROP TABLE IF EXISTS oscar_nominees

In [0]:
%sql
CREATE TABLE nominee_information(name varchar(20), amg_person_id varchar(10), top_genre varchar(10), birthday DATE, id int);

INSERT INTO nominee_information VALUES('Jennifer Lawrence','P562566','Drama','1990-08-15',755),('Jonah Hill','P418718','Comedy','1983-12-20',747),('Anne Hathaway', 'P292630','Drama', '1982-11-12',744),('Jennifer Hudson','P454405','Drama', '1981-09-12',742),('Rinko Kikuchi', 'P475244','Drama', '1981-01-06', 739);

CREATE TABLE oscar_nominees(year int, category varchar(30), nominee varchar(20), movie varchar(30), winner int, id int);

INSERT INTO oscar_nominees VALUES(2008,'actress in a leading role','Anne Hathaway','Rachel Getting Married',0,77),(2012,'actress in a supporting role','Anne HathawayLes','Mis_rables',1,78),(2006,'actress in a supporting role','Jennifer Hudson','Dreamgirls',1,711),(2010,'actress in a leading role','Jennifer Lawrence','Winters Bone',1,717),(2012,'actress in a leading role','Jennifer Lawrence','Silver Linings Playbook',1,718),(2011,'actor in a supporting role','Jonah Hill','Moneyball',0,799),(2006,'actress in a supporting role','Rinko Kikuchi','Babel',0,1253);

num_affected_rows,num_inserted_rows
7,7


In [0]:
%sql
WITH wincount AS (
  SELECT o.nominee, n.top_genre, SUM(winner) as win_cnt
  FROM oscar_nominees o 
  JOIN nominee_information n 
  ON o.nominee == n.name 
  GROUP BY o.nominee, n.top_genre
  )
SELECT top_genre FROM wincount WHERE win_cnt = (SELECT MAX(win_cnt) FROM wincount)


top_genre
Drama


In [0]:
%sql
WITH winnercount AS ( 
    SELECT nominee,
    COUNT(*) as win_count
    FROM oscar_nominees
    WHERE winner = 1
    GROUP BY nominee
)
SELECT * FROM winnercount
/*SELECT ni.top_genre 
FROM winnercount wc 
JOIN nominee_information ni 
ON wc.nominee = ni.name
ORDER BY wc.win_count LIMIT 1*/

nominee,win_count
Anne HathawayLes,1
Jennifer Lawrence,2
Jennifer Hudson,1


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql import functions as F
from pyspark.sql.functions import desc 

nominee_information_cols = ['name', 'amg_person_id', 'top_genre', 'birthday', 'id']
nominee_information_data = [('Jennifer Lawrence','P562566','Drama','1990-08-15',755),('Jonah Hill','P418718','Comedy','1983-12-20',747),('Anne Hathaway', 'P292630','Drama', '1982-11-12',744),('Jennifer Hudson','P454405','Drama', '1981-09-12',742),('Rinko Kikuchi', 'P475244','Drama', '1981-01-06', 739)]

oscar_nominees_cols = ['year', 'category', 'nominee', 'movie', 'winner', 'id']
oscar_nominees_data = [(2008,'actress in a leading role','Anne Hathaway','Rachel Getting Married',0,77),(2012,'actress in a supporting role','Anne HathawayLes','Mis_rables',1,78),(2006,'actress in a supporting role','Jennifer Hudson','Dreamgirls',1,711),(2010,'actress in a leading role','Jennifer Lawrence','Winters Bone',1,717),(2012,'actress in a leading role','Jennifer Lawrence','Silver Linings Playbook',1,718),(2011,'actor in a supporting role','Jonah Hill','Moneyball',0,799),(2006,'actress in a supporting role','Rinko Kikuchi','Babel',0,1253)]

nominee_information_df = spark.createDataFrame(data=nominee_information_data, schema=nominee_information_cols)
oscar_nominees_df = spark.createDataFrame(data=oscar_nominees_data, schema=oscar_nominees_cols)


winnercount_df = oscar_nominees_df.groupBy('nominee')\
                                .agg(F.sum('winner').alias('winner_count'))\
                                .orderBy(desc('winner_count'))
genre_wincount_df = winnercount_df\
                        .join(nominee_information_df, winnercount_df.nominee == nominee_information_df.name, "inner")\
                        .select('top_genre')
genre_wincount_df.take(1)

Out[35]: [Row(top_genre='Drama')]

In [0]:
#------Pandas------------------------------------------------------------------------------------------
import pandas as pd
nominee_information_cols = ['name', 'amg_person_id', 'top_genre', 'birthday', 'id']
nominee_information_data = [('Jennifer Lawrence','P562566','Drama','1990-08-15',755),('Jonah Hill','P418718','Comedy','1983-12-20',747),('Anne Hathaway', 'P292630','Drama', '1982-11-12',744),('Jennifer Hudson','P454405','Drama', '1981-09-12',742),('Rinko Kikuchi', 'P475244','Drama', '1981-01-06', 739)]

oscar_nominees_cols = ['year', 'category', 'nominee', 'movie', 'winner', 'id']
oscar_nominees_data = [(2008,'actress in a leading role','Anne Hathaway','Rachel Getting Married',0,77),(2012,'actress in a supporting role','Anne HathawayLes','Mis_rables',1,78),(2006,'actress in a supporting role','Jennifer Hudson','Dreamgirls',1,711),(2010,'actress in a leading role','Jennifer Lawrence','Winters Bone',1,717),(2012,'actress in a leading role','Jennifer Lawrence','Silver Linings Playbook',1,718),(2011,'actor in a supporting role','Jonah Hill','Moneyball',0,799),(2006,'actress in a supporting role','Rinko Kikuchi','Babel',0,1253)]

pd_nominee_information_df = pd.DataFrame(data=nominee_information_data, columns=nominee_information_cols) 
pd_oscar_nominees_df = pd.DataFrame(data=oscar_nominees_data, columns=oscar_nominees_cols)
pd_winnercount_df = pd_oscar_nominees_df.groupby('nominee').agg(win_count = ('winner', 'sum')).reset_index()
pd_genre_wincount_df = pd_winnercount_df.merge(pd_nominee_information_df, left_on='nominee', right_on='name', how='inner')
pd_genre_wincount_df = pd_genre_wincount_df.sort_values('win_count', ascending=False)['top_genre'].iloc[0]
pd_genre_wincount_df


Out[55]: 'Drama'

**DAY10: Amazon https://lnkd.in/gs-3sX6V Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.**

In [0]:
%sql
CREATE TABLE amazon_transactions(id int, user_id int, item varchar(15), created_at TIMESTAMP, revenue int);

INSERT INTO amazon_transactions VALUES (1,109,'milk','2020-03-03 00:00:00',123),(2,139,'biscuit','2020-03-18 00:00:00', 421), (3,120,'milk','2020-03-18 00:00:00',176), (4,108,'banana','2020-03-18 00:00:00',862), (5,130,'milk','2020-03-28 00:00:00',333), (6,103,'bread','2020-03-29 00:00:00',862), (7,122,'banana','2020-03-07 00:00:00',952), (8,125,'bread','2020-03-13 00:00:00',317), (9,139,'bread','2020-03-30 00:00:00',929), (10,141,'banana','2020-03-17 00:00:00',812), (11,116,'bread','2020-03-31 00:00:00',226), (12,128,'bread','2020-03-04 00:00:00',112), (13,146,'biscuit','2020-03-04 00:00:00',362), (14,119,'banana','2020-03-28 00:00:00',127), (15,142,'bread','2020-03-09 00:00:00',503), (16,122,'bread','2020-03-06 00:00:00',593), (17,128,'biscuit','2020-03-24 00:00:00',160), (18,112,'banana','2020-03-24 00:00:00',262), (19,149,'banana','2020-03-29 00:00:00',382), (20,100,'banana','2020-03-18 00:00:00',599);


num_affected_rows,num_inserted_rows
20,20


In [0]:
%sql
SELECT DISTINCT a.user_id
FROM amazon_transactions a JOIN amazon_transactions b 
ON a.user_id = b.user_id
AND a.created_at<b.created_at
AND date_diff(b.created_at, a.created_at) <=7



user_id
122


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import to_timestamp, col, datediff, cast
amazon_transactions_cols = ['id', 'user_id', 'item' , 'created_at', 'revenue']
amazon_transactions_data = [(1,109,'milk','2020-03-03 00:00:00',123),(2,139,'biscuit','2020-03-18 00:00:00', 421), (3,120,'milk','2020-03-18 00:00:00',176), (4,108,'banana','2020-03-18 00:00:00',862), (5,130,'milk','2020-03-28 00:00:00',333), (6,103,'bread','2020-03-29 00:00:00',862), (7,122,'banana','2020-03-07 00:00:00',952), (8,125,'bread','2020-03-13 00:00:00',317), (9,139,'bread','2020-03-30 00:00:00',929), (10,141,'banana','2020-03-17 00:00:00',812), (11,116,'bread','2020-03-31 00:00:00',226), (12,128,'bread','2020-03-04 00:00:00',112), (13,146,'biscuit','2020-03-04 00:00:00',362), (14,119,'banana','2020-03-28 00:00:00',127), (15,142,'bread','2020-03-09 00:00:00',503), (16,122,'bread','2020-03-06 00:00:00',593), (17,128,'biscuit','2020-03-24 00:00:00',160), (18,112,'banana','2020-03-24 00:00:00',262), (19,149,'banana','2020-03-29 00:00:00',382), (20,100,'banana','2020-03-18 00:00:00',599)]

amazon_transactions_df = spark.createDataFrame(data= amazon_transactions_data, schema= amazon_transactions_cols)
amazon_transactions_df = amazon_transactions_df.withColumn('created_at', to_timestamp(col('created_at')))
df1 = amazon_transactions_df.alias("df1")
df2 = amazon_transactions_df.alias("df2")

result_df = df1.join(df2, ((col("df1.user_id") == col("df2.user_id")) & \
                            (col("df1.created_at") < col("df2.created_at")) & \
                            (datediff(col("df2.created_at"), col("df1.created_at")) <= 7)))

result_df.select(col("df1.user_id")).distinct().show()

+-------+
|user_id|
+-------+
|    122|
+-------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
amazon_transactions_cols = ['id', 'user_id', 'item' , 'created_at', 'revenue']
amazon_transactions_data = [(1,109,'milk','2020-03-03 00:00:00',123),(2,139,'biscuit','2020-03-18 00:00:00', 421), (3,120,'milk','2020-03-18 00:00:00',176), (4,108,'banana','2020-03-18 00:00:00',862), (5,130,'milk','2020-03-28 00:00:00',333), (6,103,'bread','2020-03-29 00:00:00',862), (7,122,'banana','2020-03-07 00:00:00',952), (8,125,'bread','2020-03-13 00:00:00',317), (9,139,'bread','2020-03-30 00:00:00',929), (10,141,'banana','2020-03-17 00:00:00',812), (11,116,'bread','2020-03-31 00:00:00',226), (12,128,'bread','2020-03-04 00:00:00',112), (13,146,'biscuit','2020-03-04 00:00:00',362), (14,119,'banana','2020-03-28 00:00:00',127), (15,142,'bread','2020-03-09 00:00:00',503), (16,122,'bread','2020-03-06 00:00:00',593), (17,128,'biscuit','2020-03-24 00:00:00',160), (18,112,'banana','2020-03-24 00:00:00',262), (19,149,'banana','2020-03-29 00:00:00',382), (20,100,'banana','2020-03-18 00:00:00',599)]

import pandas as pd
pd_amazon_transactions_df = pd.DataFrame(data=amazon_transactions_data, columns=amazon_transactions_cols)
pd_amazon_transactions_df['created_at'] = pd.to_datetime(pd_amazon_transactions_df['created_at'])

pd_amazon_transactions_df = pd_amazon_transactions_df.sort_values(['user_id', 'created_at'])
pd_amazon_transactions_df['days_since_last_purchase'] = pd_amazon_transactions_df.groupby('user_id')['created_at'].diff().dt.days
resturning_users = pd_amazon_transactions_df.loc[pd_amazon_transactions_df['days_since_last_purchase'].between(1,7, inclusive = 'both'), 'user_id'].unique()
resturning_users


Out[13]: array([122])

**DAY11: Nvidia, Microsoft https://lnkd.in/gGPkEV35 Find the number of transactions that occurred for each product. Output the product name along with the corresponding number of transactions and order records by the product id in ascending order. You can ignore products without transactions.**

In [0]:
dbutils.fs.rm("dbfs:/user/hive/metastore/excel_sql_inventory_data", True)
dbutils.fs.rm("dbfs:/user/hive/metastore/excel_sql_transaction_data", True)

Out[19]: False

In [0]:
%sql
DROP TABLE IF EXISTS excel_sql_inventory_data;
DROP TABLE IF EXISTS excel_sql_transaction_data;

In [0]:
%sql
CREATE TABLE excel_sql_inventory_data (product_id INT,product_name VARCHAR(50),product_type VARCHAR(50),unit VARCHAR(20),price_unit FLOAT,wholesale FLOAT,current_inventory INT);

INSERT INTO excel_sql_inventory_data (product_id, product_name, product_type, unit, price_unit, wholesale, current_inventory) 
VALUES(1, 'strawberry', 'produce', 'lb', 3.28, 1.77, 13),(2, 'apple_fuji', 'produce', 'lb', 1.44, 0.43, 2),(3, 'orange', 'produce', 'lb', 1.02, 0.37, 2),(4, 'clementines', 'produce', 'lb', 1.19, 0.44, 44),(5, 'blood_orange', 'produce', 'lb', 3.86, 1.66, 19);

CREATE TABLE excel_sql_transaction_data (transaction_id INT,time TIMESTAMP,product_id INT);

INSERT INTO excel_sql_transaction_data (transaction_id, time, product_id) 
VALUES(153, '2016-01-06 08:57:52', 1),(91, '2016-01-07 12:17:27', 1),(31, '2016-01-05 13:19:25', 1),(24, '2016-01-03 10:47:44', 3),(4, '2016-01-06 17:57:42', 3),(163, '2016-01-03 10:11:22', 3),(92, '2016-01-08 12:03:20', 2),(32, '2016-01-04 19:37:14', 4),(253, '2016-01-06 14:15:20', 5),(118, '2016-01-06 14:27:33', 5);

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
SELECT 
  i.product_name, 
  COUNT(t.transaction_id) as transaction_count 
FROM excel_sql_inventory_data i 
JOIN excel_sql_transaction_data t
ON i.product_id = t.product_id
GROUP BY i.product_id, i.product_name
ORDER BY i.product_id 

product_name,transaction_count
strawberry,3
apple_fuji,1
orange,3
clementines,1
blood_orange,2


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import desc, asc, count
excel_sql_inventory_cols = ['product_id','product_name','product_type','unit','price_unit','wholesale','current_inventory']
excel_sql_inventory_data = [(1, 'strawberry', 'produce', 'lb', 3.28, 1.77, 13),(2, 'apple_fuji', 'produce', 'lb', 1.44, 0.43, 2),(3, 'orange', 'produce', 'lb', 1.02, 0.37, 2),(4, 'clementines', 'produce', 'lb', 1.19, 0.44, 44),(5, 'blood_orange', 'produce', 'lb', 3.86, 1.66, 19)]

excel_sql_transaction_cols = ['transaction_id', 'time', 'product_id']
excel_sql_transaction_data = [(153, '2016-01-06 08:57:52', 1),(91, '2016-01-07 12:17:27', 1),(31, '2016-01-05 13:19:25', 1),(24, '2016-01-03 10:47:44', 3),(4, '2016-01-06 17:57:42', 3),(163, '2016-01-03 10:11:22', 3),(92, '2016-01-08 12:03:20', 2),(32, '2016-01-04 19:37:14', 4),(253, '2016-01-06 14:15:20', 5),(118, '2016-01-06 14:27:33', 5)]

excel_sql_inventory_df = spark.createDataFrame(data=excel_sql_inventory_data, schema=excel_sql_inventory_cols)
excel_sql_transaction_df = spark.createDataFrame(data=excel_sql_transaction_data, schema=excel_sql_transaction_cols)

result_df = excel_sql_inventory_df.alias('i')\
    .join(excel_sql_transaction_df.alias('t'), col('i.product_id') == col('t.product_id'), 'inner')
result_df = result_df.groupBy('i.product_id', 'product_name').agg(count('transaction_id').alias('transaction_count'))\
    .select('product_name','transaction_count')
result_df.show()

+------------+-----------------+
|product_name|transaction_count|
+------------+-----------------+
|  strawberry|                3|
|  apple_fuji|                1|
|      orange|                3|
| clementines|                1|
|blood_orange|                2|
+------------+-----------------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
excel_sql_inventory_cols = ['product_id','product_name','product_type','unit','price_unit','wholesale','current_inventory']
excel_sql_inventory_data = [(1, 'strawberry', 'produce', 'lb', 3.28, 1.77, 13),(2, 'apple_fuji', 'produce', 'lb', 1.44, 0.43, 2),(3, 'orange', 'produce', 'lb', 1.02, 0.37, 2),(4, 'clementines', 'produce', 'lb', 1.19, 0.44, 44),(5, 'blood_orange', 'produce', 'lb', 3.86, 1.66, 19)]

excel_sql_transaction_cols = ['transaction_id', 'time', 'product_id']
excel_sql_transaction_data = [(153, '2016-01-06 08:57:52', 1),(91, '2016-01-07 12:17:27', 1),(31, '2016-01-05 13:19:25', 1),(24, '2016-01-03 10:47:44', 3),(4, '2016-01-06 17:57:42', 3),(163, '2016-01-03 10:11:22', 3),(92, '2016-01-08 12:03:20', 2),(32, '2016-01-04 19:37:14', 4),(253, '2016-01-06 14:15:20', 5),(118, '2016-01-06 14:27:33', 5)]

pd_excel_sql_inventory_df = pd.DataFrame(data=excel_sql_inventory_data, columns=excel_sql_inventory_cols)
pd_excel_sql_transaction_df = pd.DataFrame(data=excel_sql_transaction_data, columns=excel_sql_transaction_cols)

pd_result_df = pd_excel_sql_inventory_df\
    .merge(pd_excel_sql_transaction_df, left_on='product_id', right_on='product_id', how='inner')
pd_result_df = pd_result_df.groupby(['product_id', 'product_name']).agg(transaction_count = ('transaction_id', 'count')).reset_index()

pd_result_df.sort_values('product_id', ascending=True)[['product_name','transaction_count']]

,product_name,transaction_count
0,strawberry,3
1,apple_fuji,1
2,orange,3
3,clementines,1
4,blood_orange,2


**DAY12 Linkedln, Dropbox https://lnkd.in/g9tZsSmJ Write a query that calculates the difference between the highest salaries found in the marketing and engineering departments. Output just the absolute difference in salaries.**

In [0]:
%sql
CREATE TABLE db_employee (id INT,first_name VARCHAR(50),last_name VARCHAR(50),salary INT,department_id INT);

INSERT INTO db_employee (id, first_name, last_name, salary, department_id) VALUES(10306, 'Ashley', 'Li', 28516, 4),(10307, 'Joseph', 'Solomon', 19945, 1),(10311, 'Melissa', 'Holmes', 33575, 1),(10316, 'Beth', 'Torres', 34902, 1),(10317, 'Pamela', 'Rodriguez', 48187, 4),(10320, 'Gregory', 'Cook', 22681, 4),(10324, 'William', 'Brewer', 15947, 1),(10329, 'Christopher', 'Ramos', 37710, 4),(10333, 'Jennifer', 'Blankenship', 13433, 4),(10339, 'Robert', 'Mills', 13188, 1);

CREATE TABLE db_dept (id INT,department VARCHAR(50));

INSERT INTO db_dept (id, department) VALUES(1, 'engineering'),(2, 'human resource'),(3, 'operation'),(4, 'marketing');

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT ABS(
  MAX(CASE WHEN d.department = 'engineering' THEN e.salary END)-MAX(CASE WHEN d.department = 'marketing' THEN e.salary END) 
) as sal_diff
FROM db_employee e
JOIN db_dept d
ON e.department_id = d.id

sal_diff
13285


In [0]:
%sql
WITH marketing_high_salary AS (
  SELECT MAX(salary) as max_salary FROM db_employee WHERE department_id = (SELECT id FROM db_dept WHERE department= 'marketing')
),
eng_high_salary AS (
  SELECT MAX(salary) as max_salary FROM db_employee WHERE department_id = (SELECT id FROM db_dept WHERE department= 'engineering')
)
SELECT ABS((SELECT max_salary FROM marketing_high_salary) - (SELECT max_salary FROM eng_high_salary)) as sal_diff


sal_diff
13285


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import col, max, abs
db_employee_cols = ['id','first_name','last_name','salary','department_id']
db_employee_data = [(10306, 'Ashley', 'Li', 28516, 4),(10307, 'Joseph', 'Solomon', 19945, 1),(10311, 'Melissa', 'Holmes', 33575, 1),(10316, 'Beth', 'Torres', 34902, 1),(10317, 'Pamela', 'Rodriguez', 48187, 4),(10320, 'Gregory', 'Cook', 22681, 4),(10324, 'William', 'Brewer', 15947, 1),(10329, 'Christopher', 'Ramos', 37710, 4),(10333, 'Jennifer', 'Blankenship', 13433, 4),(10339, 'Robert', 'Mills', 13188, 1)]


db_dept_cols = ['id', 'department']
db_dept_data = [(1, 'engineering'),(2, 'human resource'),(3, 'operation'),(4, 'marketing')]

db_employee_df = spark.createDataFrame(data=db_employee_data, schema=db_employee_cols)
db_dept_df = spark.createDataFrame(data=db_dept_data, schema=db_dept_cols)

result_df = db_employee_df.join(db_dept_df, db_employee_df.department_id == db_dept_df.id, 'inner')
marketing_high_sal_df = result_df.filter(result_df.department == 'marketing').select(max(col('salary')).alias('max_sal'))
marketing_high_sal = marketing_high_sal_df.collect()[0]['max_sal']
eng_high_sal_df = result_df.filter(result_df.department == 'engineering').select(max(col('salary')).alias('max_sal'))
eng_high_sal = eng_high_sal_df.collect()[0]['max_sal']
sal_diff = marketing_high_sal - eng_high_sal
print(sal_diff)

# Without using collect operation
result_df = marketing_high_sal_df.crossJoin(eng_high_sal_df)
result_df = result_df.withColumn('sal_diff', abs(marketing_high_sal_df.max_sal - eng_high_sal_df.max_sal)).select('sal_diff')
result_df.show()

13285
+--------+
|sal_diff|
+--------+
|   13285|
+--------+

